# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [11]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
from pathlib import Path

# Import API key
from api_keys import geoapify_key

In [16]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
csv_file = Path("../output_data/cities.csv")
city_data_df = pd.read_csv(csv_file)

# Display sample data
city_data_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,puerto natales,-51.7236,-72.4875,4.44,89,89,2.68,CL,1684037174
1,1,lorengau,-2.0226,147.2712,30.27,65,13,2.43,PG,1684037175
2,2,avarua,-21.2078,-159.7750,23.03,94,75,2.06,CK,1684037176
3,3,wailua homesteads,22.0669,-159.3780,24.24,86,100,5.14,US,1684037176
4,4,adamstown,-25.0660,-130.1015,23.22,72,100,8.94,PN,1684037177
...,...,...,...,...,...,...,...,...,...,...
582,582,pleven,43.4167,24.6167,11.40,78,100,2.35,BG,1684037570
583,583,ulsteinvik,62.3432,5.8487,10.97,91,98,0.45,NO,1684037571
584,584,qashar,53.3906,62.9297,7.88,62,60,3.79,KZ,1684037571
585,585,aimores,-19.4958,-41.0639,19.99,94,100,1.13,BR,1684037572


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [19]:
%%capture --no-display

# Configure the map
map_plot_1 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 600,
    frame_height = 400,
    size = "Humidity",
    scale = 0.5,
    color = "City",
    hover_cols = ['City', 'Humidity']
)

# Display the map plot
map_plot_1

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [28]:
# Narrow down cities that fit criteria and drop any results with null values
clean_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <27) & (city_data_df["Max Temp"] > 21) & (city_data_df["Wind Speed"] < 5) & (city_data_df["Cloudiness"] == 0)].dropna()

# Drop any rows with null values
clean_cities_df = clean_cities_df.dropna()

# Display sample data
clean_cities_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
116,116,swan hill,-35.3378,143.5544,22.26,34,0,2.08,AU,1684037031
323,323,santa rosalia,27.3167,-112.2833,21.87,48,0,2.78,MX,1684037398
403,403,douglas,41.3000,-96.1670,21.99,77,0,1.54,US,1684037294
466,466,shymkent,42.3000,69.6000,24.33,33,0,1.00,KZ,1684037494
469,469,urumqi,43.8010,87.6005,25.53,24,0,3.00,CN,1684037496


### Step 3: Create a new DataFrame called `hotel_df`.

In [29]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df= clean_cities_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ''

# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
116,swan hill,AU,-35.3378,143.5544,34,
323,santa rosalia,MX,27.3167,-112.2833,48,
403,douglas,US,41.3000,-96.1670,77,
466,shymkent,KZ,42.3000,69.6000,33,
469,urumqi,CN,43.8010,87.6005,24,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [30]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories":"accommodation",
    "limit":20,
    "apiKey":geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = hotel_df.loc[index, "Lat"]
    lng = hotel_df.loc[index, "Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f'circle:{lng},{lat},{radius}'
    params["bias"] = f'proximity:{lng},{lat}'
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
swan hill - nearest hotel: No hotel found
santa rosalia - nearest hotel: No hotel found
douglas - nearest hotel: No hotel found
shymkent - nearest hotel: No hotel found
urumqi - nearest hotel: No hotel found
coahuayana de hidalgo - nearest hotel: No hotel found
kuraymah - nearest hotel: No hotel found
tamanrasset - nearest hotel: No hotel found
felipe carrillo puerto - nearest hotel: No hotel found


,City,Country,Lat,Lng,Humidity,Hotel Name
116,swan hill,AU,-35.3378,143.5544,34,No hotel found
323,santa rosalia,MX,27.3167,-112.2833,48,No hotel found
403,douglas,US,41.3000,-96.1670,77,No hotel found
466,shymkent,KZ,42.3000,69.6000,33,No hotel found
469,urumqi,CN,43.8010,87.6005,24,No hotel found
495,coahuayana de hidalgo,MX,18.7000,-103.6583,73,No hotel found
509,kuraymah,SD,18.5500,31.8500,19,No hotel found
532,tamanrasset,DZ,22.7850,5.5228,38,No hotel found
579,felipe carrillo puerto,MX,19.5833,-88.0500,86,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [31]:
%%capture --no-display

# Configure the map plot
hotel_map = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo= True,
    tiles="OSM",
    size ='Humidity',
    scale= 1,
    alpha = 0.5,
    frame_width = 700,
    frame_height = 500,
    hover_cols = ["Hotel Name", "Country"]
    
)

# Display the map
hotel_map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (Humidity,Hotel Name,Country)